# 第6回講義 宿題

## 課題
RNNを用いてIMDbのsentiment analysisを実装してみましょう．

ネットワークの形などに制限はとくになく，今回のLessonで扱った内容以外の工夫も組み込んでもらって構いません．

## 目標値
F値：0.85

## ルール
- 以下のセルで指定されている`x_train`, `t_train`以外の学習データは使わないでください．

## 提出方法
- 2つのファイルを提出していただきます．
  1. テストデータ `x_test` に対する予測ラベルを`submission_pred.csv`として保存し，Omnicampusの宿題から「第6回 回帰結合型ニューラルネットワーク」を選択して提出してください．
  2. それに対応するpythonのコードを`submission_code.py`として保存し，Omnicampusの宿題から「第6回 回帰結合型ニューラルネットワーク (code)」を選択して提出してください．
    - セルに書いたコードを.py形式で保存するためには%%writefileコマンドなどを利用してください．
    - writefileコマンドではファイルの保存のみが行われセル内のpythonコード自体は実行されません．そのため，実際にコードを走らせる際にはwritefileコマンドをコメントアウトしてください．


- コードの内容を変更した場合は，1と2の両方を提出し直してください．

- なお採点は1で行い，2はコードの確認用として利用します．(成績優秀者はコード内容を公開させていただくかもしれません)


## 評価方法

- 予測ラベルの`t_test`に対するF値で評価します．
- 即時採点しLeader Boardを更新します．（採点スケジュールは別アナウンス）
- 締切時の点数を最終的な評価とします．



## データの読み込み（このセルは修正しないでください）

In [2]:
import random
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchtext import datasets
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from collections import Counter
import pandas as pd
import string
import re
from typing import List, Union
import torchtext

torchtext.disable_torchtext_deprecation_warning()

seed = 1234
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)


# 学習データ
data_dir = '../data'
x_train = np.load(os.path.join(data_dir, 'x_train.npy'), allow_pickle=True)
t_train = np.load(os.path.join(data_dir, 't_train.npy'), allow_pickle=True)

# 検証データを取る
x_train, x_valid, t_train, t_valid = train_test_split(x_train, t_train, test_size=0.2, random_state=seed)
    
# テストデータ
x_test = np.load(os.path.join(data_dir, 'x_test.npy'), allow_pickle=True)


def text_transform(text: List[int], max_length=256):
    # <BOS>はすでに1で入っている．<EOS>は2とする．
    text = text[:max_length - 1] + [2]

    return text, len(text)

def collate_batch(batch):
    label_list, text_list, len_seq_list = [], [], []
    
    for sample in batch:
        if isinstance(sample, tuple):
            label, text = sample

            label_list.append(label)
        else:
            text = sample.copy()
            
        text, len_seq = text_transform(text)
        text_list.append(torch.tensor(text))
        len_seq_list.append(len_seq)
        
    # NOTE: 宿題用データセットでは<PAD>は3です．
    return torch.tensor(label_list), pad_sequence(text_list, padding_value=3).T, torch.tensor(len_seq_list)


word_num = np.concatenate(np.concatenate((x_train, x_test))).max() + 1
print(f"単語種数: {word_num}")

単語種数: 88587


In [12]:
len(x_train[0]), len(x_train[1]), len(x_train[2])

(773, 117, 580)

In [9]:
print("Shape of x_train:", x_train.shape)
print("Shape of t_train:", t_train.shape)
print("Shape of x_valid:", x_valid.shape)
print("Shape of t_valid:", t_valid.shape)
print("Shape of x_test:", x_test.shape)

print(np.concatenate((x_train, x_test)).shape)
print(np.concatenate(np.concatenate((x_train, x_test))).shape)

Shape of x_train: (32000,)
Shape of t_train: (32000,)
Shape of x_valid: (8000,)
Shape of t_valid: (8000,)
Shape of x_test: (10000,)
(42000,)
(9847205,)


## 実装

In [13]:
batch_size = 128

train_dataloader = DataLoader(
    [(t, x) for t, x in zip(t_train, x_train)],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_batch,
)
valid_dataloader = DataLoader(
    [(t, x) for t, x in zip(t_valid, x_valid)],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_batch,
)
test_dataloader = DataLoader(
    x_test,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_batch,
)

In [ ]:
def torch_log(x):
    return torch.log(torch.clamp(x, min=1e-10))


class Embedding(nn.Module):
    # WRITE ME


class SequenceTaggingNet(nn.Module):
    # WRITE ME

In [ ]:
emb_dim = 100
hid_dim = 50
n_epochs = 10
device = 'cuda'

net = SequenceTaggingNet(word_num, emb_dim, hid_dim)
net.to(device)
optimizer = optim.Adam(net.parameters())

for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []

    net.train()
    n_train = 0
    acc_train = 0
    for label, line, len_seq in train_dataloader:
        
        # WRITE ME

        losses_train.append(loss.tolist())

        n_train += t.size()[0]

    # Valid
    t_valid = []
    y_pred = []
    net.eval()
    for label, line, len_seq in valid_dataloader:

        # WRITE ME

        t_valid.extend(t.tolist())
        y_pred.extend(pred.tolist())

        losses_valid.append(loss.tolist())

    print('EPOCH: {}, Train Loss: {:.3f}, Valid Loss: {:.3f}, Validation F1: {:.3f}'.format(
        epoch,
        np.mean(losses_train),
        np.mean(losses_valid),
        f1_score(t_valid, y_pred, average='macro')
    ))

In [ ]:
net.eval()

y_pred = []
for _, line, len_seq in test_dataloader:

    x = line.to(device)
    len_seq.to(device)

    h = net(x, torch.max(len_seq), len_seq)
    y = torch.sigmoid(h).squeeze()

    pred = y.round().squeeze()  # 0.5以上の値を持つ要素を正ラベルと予測する

    y_pred.extend(pred.tolist())


submission = pd.Series(y_pred, name='label')
submission.to_csv('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture06/submission_pred.csv', header=True, index_label='id')